## Importing libraries 

In [ ]:
import pandas as pd
import numpy as np
import bcrypt
from pymongo import MongoClient
import ssl
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import re
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

## Importing and pre-processing dataset 

In [ ]:
data=pd.read_csv('merged_df.csv')

In [ ]:
data.drop(['Unnamed: 0','_id','password','Timestamp'],axis=1,inplace=True)

In [ ]:
data

In [ ]:
# Disable SSL verification
ssl._create_default_https_context = ssl._create_unverified_context

nltk.download('stopwords')

In [ ]:
#Listing out stopwords in english
x=stopwords.words('english')

In [ ]:
# Download required NLTK data
nltk.download('wordnet')
nltk.download('omw-1.4')

def general_preprocess(to_preprocess_column_name, df_name):
    to_preprocess_column_name_processed = []
    lemma = WordNetLemmatizer()
    
    for i in range(len(df_name)):
        col = df_name.iloc[i][to_preprocess_column_name]
        col = re.sub('[^a-zA-Z]', ' ', col)  # Remove non-alphabetic characters
        col = col.lower()  # Convert to lowercase
        col = col.split()  # Split into words
        col = [lemma.lemmatize(word) for word in col if word not in set(stopwords.words('english'))]  # Lemmatize and remove stopwords
        col = ' '.join(col)  # Join words back into a single string
        to_preprocess_column_name_processed.append(col)
    
    return to_preprocess_column_name_processed

In [ ]:
Topic_preprocessed=general_preprocess('Topic',data)

In [ ]:
Topic_preprocessed

In [ ]:
Skills_preprocessed=general_preprocess('Skills',data)

In [ ]:
Skills_preprocessed

## Combining target columns

In [ ]:
df_preprocessed = pd.DataFrame({
    'Skills': Skills_preprocessed,
    'Topic': Topic_preprocessed
})

In [ ]:
df_preprocessed['Combined'] = df_preprocessed['Skills'] + ', ' + df_preprocessed['Topic']

# Drop the individual columns if you only need the combined column
df_combined = df_preprocessed.drop(columns=['Skills', 'Topic'])

In [ ]:
df_combined

## BERT - large uncased

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')  # You can change this to other BERT models
model = BertModel.from_pretrained('bert-large-uncased')

In [ ]:
def get_embeddings(texts):
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        # Use the mean of token embeddings as the sentence embedding
        sentence_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        embeddings.append(sentence_embedding)
    return embeddings

In [ ]:
embeddings = get_embeddings(df_combined['Combined'])

In [ ]:
embeddings = np.array(embeddings)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute the cosine similarity matrix
cosine_sim_matrix = cosine_similarity(embeddings)

## Clustering 

In [ ]:
from sklearn.cluster import AffinityPropagation
# Step 3: Apply Affinity Propagation
affinity_propagation = AffinityPropagation(affinity='precomputed', random_state=42)
labels = affinity_propagation.fit_predict(cosine_sim_matrix)

# Step 4: Create a DataFrame to view results
df = pd.DataFrame({'Skillset': train_data, 'Cluster': labels})

# Print the DataFrame with clusters
print("Clustered Skillsets with Affinity Propagation:")
print(df)

# Print the number of items in each cluster
cluster_counts = df['Cluster'].value_counts().sort_index()
print("\nNumber of Items in Each Cluster:")
print(cluster_counts)

In [ ]:
#Printing the number of data in each cluster
cluster_counts = df['Cluster'].value_counts().sort_index()
print("\nNumber of Items in Each Cluster:")
print(cluster_counts)

## Dimensionality reduction and visualization

In [ ]:
# Apply t-SNE to the BERT embeddings
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
reduced_matrix = tsne.fit_transform(embeddings_np)

# Create a DataFrame with t-SNE results and cluster labels
df_tsne = pd.DataFrame(reduced_matrix, columns=['TSNE1', 'TSNE2'])
df_tsne['Cluster'] = labels  # Affinity Propagation cluster labels
#df_tsne['Skillset'] = train_data  # Optional: Include original skillset data

# Plot the clusters
plt.figure(figsize=(12, 8))
sns.scatterplot(x='TSNE1', y='TSNE2', hue='Cluster', data=df_tsne, palette='viridis', marker='o', s=100)
plt.title('Clusters Visualized with t-SNE')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.legend(title='Cluster')
plt.savefig('affinity_propagation_clusters.png', dpi=600, bbox_inches='tight')
plt.show()

## Manually obsering Skillset similarity 

In [ ]:
cluster_df = df[df['Cluster'] == 60]

In [ ]:
cluster_df

## Intra-cluster similarity 

In [ ]:
intra_cluster_similarities = []

# Get unique cluster labels
unique_clusters = np.unique(labels)

for cluster in unique_clusters:
    # Get the indices of the data points in the current cluster
    cluster_indices = np.where(labels == cluster)[0]
    
    if len(cluster_indices) > 1:  # Only compute if more than one point
        # Extract the embeddings for the current cluster
        cluster_embeddings = embeddings[cluster_indices]

        # Compute the cosine similarity matrix for the current cluster
        cluster_sim_matrix = cosine_similarity(cluster_embeddings)
        
        # Get the number of points in the cluster
        num_points = len(cluster_indices)
        
        # Compute the average similarity within the cluster
        # Exclude the diagonal (self-similarity) from the average
        avg_similarity = (np.sum(cluster_sim_matrix) - num_points) / (num_points * (num_points - 1))
        
        intra_cluster_similarities.append(avg_similarity)
    else:
        # Not enough points to calculate
        intra_cluster_similarities.append(np.nan)

# Calculate the overall intra-cluster similarity
# Filter out NaN values (clusters with only one point)
valid_similarities = [sim for sim in intra_cluster_similarities if not np.isnan(sim)]
overall_intra_cluster_similarity = np.mean(valid_similarities) if valid_similarities else np.nan

# Print the overall intra-cluster similarity
print(f"Overall Intra-Cluster Similarity: {overall_intra_cluster_similarity:.4f}")

## Sillhouette Score 

In [ ]:
from sklearn.metrics import silhouette_score

# Assuming `embeddings` is your feature matrix and `labels` are your cluster labels
silhouette_avg = silhouette_score(embeddings, labels)

# Print the Silhouette Score
print(f"Silhouette Score: {silhouette_avg:.4f}")


## Davies-Bouldin Index

In [ ]:
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics import davies_bouldin_score

# Compute the Davies-Bouldin Index
dbi = davies_bouldin_score(embeddings, labels)

# Print the Davies-Bouldin Index
print(f"Davies-Bouldin Index: {dbi:.4f}")


In [ ]:
data['Combined'] = df['Skillset']
data['Cluster']=df['Cluster']

In [ ]:
data

## Recommendation function 

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def recommend_profiles_by_topic_skills(user_index, df, top_n=5):
    """
    Recommends the top N profiles to a target profile based on 'Topic' and 'Skills' columns.

    Parameters:
    - user_index (int): The index of the target profile in the DataFrame.
    - df (pandas.DataFrame): The DataFrame containing profile data.
    - top_n (int): The number of top profiles to recommend.

    Returns:
    - list of tuples: Each tuple contains the index and similarity score of a recommended profile.
    """
    
    bert_embeddings = get_embeddings(data['Combined']) 
    bert_embeddings = np.array(bert_embeddings)
    
    # Get the TF-IDF vector for the target profile
    target_vector = bert_embeddings[user_index].reshape(1, -1)
    
    # Compute cosine similarities between the target profile and all other profiles
    similarities = cosine_similarity(target_vector, bert_embeddings).flatten()
    
    # Exclude the target profile itself by setting its similarity score to -1
    similarities[user_index] = -1
    
    # Get the indices of the top N most similar profiles
    top_indices = similarities.argsort()[-top_n:][::-1]
    
    # Create a list of tuples with index and similarity score
    recommendations = [(idx, similarities[idx]) for idx in top_indices]
    
    return recommendations

def print_profile_details(profile_index, df):
    """
    Prints the details of a specific profile.

    Parameters:
    - profile_index (int): The index of the profile in the DataFrame.
    - df (pandas.DataFrame): The DataFrame containing profile data.
    """
    profile_details = df.iloc[profile_index]
    print(f"Details of Target Profile (Index: {profile_index}):")
    print(f"Name: {profile_details['Name']}")
    print(f"Email: {profile_details['Email']}")
    print(f"Skills: {profile_details['Skills']}")
    print(f"Domain Interest: {profile_details['Topic']}")
    print(f"Cluster: {profile_details['Cluster']}")
    print("-" * 40)


def print_recommendation_details(recommendations, df):
    """
    Prints the details of the recommended profiles.

    Parameters:
    - recommendations (list of tuples): Each tuple contains the index and similarity score of a recommended profile.
    - df (pandas.DataFrame): The DataFrame containing profile data.
    """
    for idx, score in recommendations:
        profile_details = data.iloc[idx]
        print(f"Index: {idx}")
        print(f"Similarity Score: {score:.4f}")
        print(f"Name: {profile_details['Name']}")
        print(f"Email: {profile_details['Email']}")
        print(f"Skills: {profile_details['Skills']}")
        print(f"Domain Interest: {profile_details['Topic']}")
        print(f"Cluster: {profile_details['Cluster']}")
        print("-" * 40)
        
target_profile_index = 575  # Replace with the index of the target profile
print_profile_details(target_profile_index, data)
top_n_recommendations = recommend_profiles_by_topic_skills(target_profile_index, data, top_n=5)
print_recommendation_details(top_n_recommendations, data)


In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

def generate_keyword_cloud(df, column, title):
    """
    Generates and plots a word cloud for a specific column in the DataFrame.

    Parameters:
    - df (pandas.DataFrame): The DataFrame containing profile data.
    - column (str): The column name for which the word cloud is to be generated.
    - title (str): The title for the word cloud plot.
    """
    # Combine all text in the specified column
    text = " ".join(df[column].dropna().astype(str).values)
    
    # Generate the word cloud
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    
    # Plot the word cloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title, fontsize=20)
    plt.axis('off')
    plt.savefig('keyword_cloud.png', dpi=600, bbox_inches='tight')
    plt.show()

# Generate and plot the keyword cloud for 'Skills'
generate_keyword_cloud(data, 'Skills', 'Keyword Cloud for recommendations')